In [ ]:
import scipy.io
import numpy as np 
from numpy import random
import matplotlib.pyplot as plt

In [ ]:
def lire_alpha_digit(indices):

    data = scipy.io.loadmat("binaryalphadigs")

    images = []
    for idx in indices:
        for i in range(len(data['dat'][idx])):
            image = data['dat'][idx][i]
            image = np.array(image).flatten()
            images.append(image)

    return np.array(images)

In [22]:
class RBM:
    def __init__(self, p, q):
        """Constructeur de la classe"""
        self.p = p
        self.q = q
        self.a = np.zeros(p)
        self.b = np.zeros(q)
        self.w = np.random.normal(0, 1, (p,q)) * 0.01

    def entree_sortie(self, X):
        H = 1 / (1 + np.exp(-np.dot(X,self.w) - self.b))
        return H
    
    def sortie_entree(self, H):
        X = 1 / (1 + np.exp(-np.dot(H, self.w.T) - self.a ))
        return X
    
    def train(self, X, nb_epochs, taille_batch, epsilon):
        erreurs = []
        n = X.shape[0]
        for i in range(nb_epochs):
            np.random.shuffle(X)
            for j in range(0, n, taille_batch):
                X_batch = X[j:min(j + taille_batch, n), :]
                t_b = X_batch.shape[0]
                V_0 = X_batch
                phv0 = self.entree_sortie(X_batch)
                h0 = (np.random.rand(t_b, self.q) < phv0) * 1
                pv1 = self.sortie_entree(h0)
                V_1 = (np.random.rand(t_b, self.p) < pv1) * 1
                phv1 = self.entree_sortie(V_1)
                grad_a = np.sum(X_batch - V_1, axis=0)
                grad_b = np.sum(phv0 - phv1, axis=0)
                grad_w = np.dot(X_batch.T, phv0) - np.dot(V_1.T, phv1)
                
                self.w += (epsilon / t_b) * grad_w
                self.a += (epsilon / t_b) * grad_a
                self.b += (epsilon / t_b) * grad_b

            H = self.entree_sortie(X)
            X_rec = self.sortie_entree(H)
            erreur = np.sum((X-X_rec)**2) / (n * self.p)
            erreurs.append(erreur)
        return erreurs
    
    def generer_image(self, num_iterations=1000, nb_images=1):
        """Générer une image en utilisant un échantillonneur de Gibbs."""
        images = np.zeros((nb_images, self.p))
        for i in range(nb_images):
            np.random.seed(None)
            v = np.random.rand(1, self.p)
            for _ in range(num_iterations):
                h = self.entree_sortie(v)
                v = self.sortie_entree(h)
            images[i,:] = v.flatten()
        return images

In [ ]:
if __name__ == "__main__":

    p = 320  # taille de l'image (=20*16)
    q = 64 # nb d'unités cachées (hyper paramètre)
    
    rbm = RBM(p, q)
    
    nb_caracteres_1 = 1
    indices = [10]
    data = lire_alpha_digit(indices)
    
    erreurs = rbm.train(data, nb_epochs=100, taille_batch=10, epsilon=0.1)

    nb_images=6
    generated_image = rbm.generer_image(nb_images=nb_images)
    
    nb_affichages = 3
    if nb_images >= nb_affichages:
        for i in range(nb_affichages):
            plt.imshow(generated_image[i,:].reshape(20, 16), cmap='gray')
            plt.title(f"Image générée pour {nb_caracteres_1} caractères appris")
            plt.show()

    plt.plot(erreurs)
    plt.title("Erreur de reconstruction")
    plt.xlabel("Epoch")
    plt.ylabel("Erreur de reconstruction")
    plt.legend()
    plt.show()